In [8]:
import pandas as pd
import numpy as np

In [9]:
# dataframe livros do dia
df = pd.read_json("./out/books-list-2021-08-27-t10-21-48-326-z.json")

In [10]:
#organizar datas e colunas, calcular descontos e remover duplicados

df = df.assign(dia=df['livro_do_dia_datas']).explode('dia').reset_index(drop=True)

df["desconto_livrdia_euro"] = df["pvp"] - df["pvp_livro_do_dia"]
df["desconto_livrdia_perc"] = (df["desconto_livrdia_euro"] / df["pvp"]) * 100

df = df.drop_duplicates(subset=['isbn', 'titulo', 'autor', 'chancela', 'dia'], keep='first')

In [11]:
#Está no Plano Nacional de Leitura?

pnl = pd.read_csv("isbn_pnl_sem_duplicados.csv", index_col=0)
pnl     

pnl["ISBN"] = pnl["ISBN"].str.replace("-", "", regex=True)
isbn_pnl = list(set(pnl["ISBN"]).intersection(df["isbn"]))
df['PNL'] = np.where(df['isbn'].isin(isbn_pnl), '✅', '')


In [24]:
# organizar colunas e exportar

df = df[['dia', 'isbn','PNL', 'titulo', 'autor', 'chancela', 'pvp', 'pvp_livro_do_dia', 'desconto_livrdia_perc', 'participant_name', 'stand']]
df = df.apply(lambda x: x.astype(str).str.title())
df.to_csv("livros-do-dia-limpo.csv", index=False)